In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.distributions as D
from functools import partial
import os
import json
from copy import deepcopy

from pinf.plot.utils import eval_pdf_on_grid_2D
from pinf.models.GMM import GMM
from pinf.datasets.gradients import dS_dbeta_2D_GMM

Settings

---

In [ ]:
generate_new = True
n_samples = 500000
bs = 10000

Initialize the target distribution

---

In [ ]:
means = torch.tensor([
    [-1.0,2.0],
    [3.0,7.0],
    [-4.0,2.0],
    [-2.0,-4.0],
    [0.0,4.0],
    [5.0,-2.0]
])

#Covariance matrices
S = torch.tensor([
        [[ 0.2778,  0.4797],
         [ 0.4797,  0.8615]],

        [[ 0.8958, -0.0249],
         [-0.0249,  0.1001]],

        [[ 1.3074,  0.9223],
         [ 0.9223,  0.7744]],

        [[ 0.0305,  0.0142],
         [ 0.0142,  0.4409]],

        [[ 0.0463,  0.0294],
         [ 0.0294,  0.3441]],
        
        [[ 0.15,  0.0294],
         [ 0.0294,  1.5]]])

p_target = GMM(means = means,covs=S)

In [ ]:
def p_beta(x,beta,Z = None):
    q_beta = p_target(x).pow(beta)

    if Z is None:
        return q_beta
    
    else:
        return q_beta / Z

Approximate the partition function of the power-scaled distribution

---

In [ ]:
T_list = torch.linspace(np.log(0.1),np.log(10),20).exp()
T_list = torch.cat((T_list,torch.tensor([1.0])))
T_list = T_list.sort().values


if generate_new:

  fig,axes = plt.subplots(1,len(T_list),figsize = (30,15))

  Z_T_list = []

  for i,T in enumerate(T_list):
      p = partial(p_beta,beta = 1 / round(T_list[i].item(),5))

      pdf_grid,x_grid,y_grid = eval_pdf_on_grid_2D(
        pdf=p,
        x_lims = [-15,15],
        y_lims = [-15,15],
        x_res = 10000,
        y_res = 10000,
        )
      
      # Get the volume element
      dA = (x_grid[0,1] - x_grid[0,0]) * (y_grid[1,0] - y_grid[0,0])

      # Get the partition function
      Z_T = pdf_grid.sum() * dA
      Z_T_list.append(Z_T)
      print(Z_T)
      
      axes[i].imshow(pdf_grid,extent = [x_grid.min(),x_grid.max(),y_grid.min(),y_grid.max()],origin = 'lower')

      axes[i].set_title(f'T = {round(T_list[i].item(),5)}')
      axes[i].axis('off')

In [ ]:
folder = "../data/2D_GMM"

if generate_new:

    if not os.path.exists(folder):
        os.makedirs(folder)

    dict_Z_T = {
    }

    for i in range(len(T_list)):
        dict_Z_T[f"{round(T_list[i].item(),5)}"] = Z_T_list[i].item()

    with open(os.path.join(folder,'Z_T.json'), 'w') as f:
        json.dump(dict_Z_T, f)
    f.close()

Perform rejection sampling

---

In [ ]:
class ConcatenatedGMM(GMM):
    def __init__(self,means:torch.tensor,covs:torch.tensor,sigma_noise:float,weights:torch.tensor = None)->None:

        #Compute new covariance matrices

        for i in range(len(means)):
            
            covs[i] = covs[i] + torch.eye(2) * sigma_noise**2
        
        super().__init__(means = means,covs = covs,weights = weights)

In [ ]:
if generate_new:

    if not os.path.exists(os.path.join(folder,"validation_data/")):
        os.makedirs(os.path.join(folder,"validation_data/"))

    if not os.path.exists(os.path.join(folder,"training_data/")):
        os.makedirs(os.path.join(folder,"training_data/"))

    for i in range(len(T_list)):

        if T_list[i] > 1.0:
            p_prop = ConcatenatedGMM(means = means,covs = deepcopy(S),sigma_noise = 2.0)
        else:
            p_prop = GMM(means = means,covs = deepcopy(S))

        def p_beta(x,beta,Z = None):
            p_GMM_plain = GMM(means = means,covs = deepcopy(S))
            q_beta = p_GMM_plain(x).pow(beta)

            if Z is None:
                return q_beta
            
            else:
                return q_beta / Z

        p_eval = partial(p_beta,beta = 1 / round(T_list[i].item(),5),Z = Z_T_list[i])

        samples_i = torch.zeros([0,2])

        while True:

            #Get u
            u = torch.rand(bs)

            #get proposals
            x_prop = p_prop.sample(bs)

            r = p_eval(x_prop) / (p_prop(x_prop) * 10)

            accept = u < r

            samples_i = torch.cat((samples_i,x_prop[accept]),dim = 0)
           
            if len(samples_i) > n_samples:
                break
        
        torch.save(samples_i[:int(0.8 * n_samples)],os.path.join(folder,f'training_data/T_{round(T_list[i].item(),5)}_dim_2.pt'))
        torch.save(samples_i[int(0.8 * n_samples):],os.path.join(folder,f'validation_data/T_{round(T_list[i].item(),5)}_dim_2.pt'))

Compare the empirical distribution to the target distribution 

---

In [ ]:
fig,axes = plt.subplots(2,len(T_list),figsize = (len(T_list) * 15,2 * 15))
with open(os.path.join(folder,'Z_T.json'), 'r') as f:
  dict_Z_T = json.load(f)
f.close()

for i,T in enumerate(T_list):

    T_i = round(T.item(),5)

    #Ground truth distribution
    p = partial(p_beta,beta = 1 / round(T_list[i].item(),5), Z = dict_Z_T[f"{round(T_list[i].item(),5)}"])

    pdf_grid,x_grid,y_grid = eval_pdf_on_grid_2D(
        pdf=p,
        x_lims = [-15,15],
        y_lims = [-15,15],
        x_res = 150,
        y_res = 150,
    )
      
    axes[0][i].imshow(pdf_grid,extent = [x_grid.min(),x_grid.max(),y_grid.min(),y_grid.max()],origin = 'lower')

    axes[0][i].set_title(f'T = {round(T_list[i].item(),5)}')
    axes[0][i].axis('off')
    axes[0][i].set_title(T_i)


    #Empirical distribution based on the samples
    data_val_i = torch.load(os.path.join(folder,f'training_data/T_{T_i}_dim_2.pt'))

    x = data_val_i[:,0].numpy()
    y = data_val_i[:,1].numpy()

    h,_, _, image = axes[1][i].hist2d(x,y,bins = 150, density = True,range = [[-15,15],[-15,15]])

Compute the expectation value of the energy

---

In [ ]:
beta_eval = torch.linspace(0.5,2.0,200)
res_expectation_val = 1500
lim_expectation_approx = 25
EX_A_storage = []

pdf_grid,x_grid,y_grid = eval_pdf_on_grid_2D(
    pdf=p_target,
    x_lims = [-lim_expectation_approx,lim_expectation_approx],
    y_lims = [-lim_expectation_approx,lim_expectation_approx],
    x_res = res_expectation_val,
    y_res = res_expectation_val,
    )

dV = (x_grid[0,1] - x_grid[0,0]) * (y_grid[1,0] - y_grid[0,0])

dS_dparam = partial(dS_dbeta_2D_GMM,gmm = p_prop,beta = 1.0,device = "cpu")

A,_,_ = eval_pdf_on_grid_2D(
        pdf= dS_dparam,
        x_lims = [-lim_expectation_approx,lim_expectation_approx],
        y_lims = [-lim_expectation_approx,lim_expectation_approx],
        x_res = res_expectation_val,
        y_res = res_expectation_val,
        )

for i,beta_i in enumerate(beta_eval):
    
    #Get the partition function
    Z_i = pdf_grid.pow(beta_i).sum() * dV
    EX_A = (A * pdf_grid.pow(beta_i) * dV / Z_i).sum()

    EX_A_storage.append(EX_A)

data = np.concatenate((beta_eval.reshape(-1,1),np.array(EX_A_storage).reshape(-1,1)),1)

np.savetxt(os.path.join(folder,"EX_A_ground_truth.txt"),data,header = "beta\tEX_S")    

In [ ]:
plt.plot(beta_eval,EX_A_storage)